In [303]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn import preprocessing
import xgboost
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import GridSearchCV

In [304]:
df=pd.read_csv("train_u6lujuX_CVtuZ9i.csv")

In [305]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [338]:
def pre_processing(data_frame,train=True):
    data_frame.Dependents.replace(['3+'],[3],inplace=True)
    data_frame['Gender'].fillna('No',inplace=True)
    data_frame['Married'].fillna('UNKNOWN',inplace=True)
    data_frame['Education'].fillna('UNKNOWN',inplace=True)
    data_frame['Self_Employed'].fillna('UNKNOWN',inplace=True)
    data_frame['Property_Area'].fillna('UNKNOWN',inplace=True)
    data_frame['Dependents'].fillna(-1,inplace=True)
    data_frame['Credit_History'].fillna(0,inplace=True)
    data_frame.fillna(-1,inplace=True)
    data_frame['Dependents']=data_frame['Dependents'].astype('Float64')

    data_frame['IncomePerDep']=data_frame['ApplicantIncome']/(data_frame['Dependents'] + 2)
    data_frame['LoanByIncome']=(data_frame['LoanAmount']*1000)/data_frame['ApplicantIncome']
    data_frame['LoanPerYear']=(data_frame['LoanAmount']*1000)/(data_frame['Loan_Amount_Term']/12)
    data_frame['Residual']=data_frame['ApplicantIncome']+data_frame['CoapplicantIncome']-data_frame['LoanPerYear']
    data_frame['IncomeRatio']=(data_frame['ApplicantIncome']+data_frame['CoapplicantIncome'])/(data_frame['LoanAmount']+2)
    if train:
        data_frame.drop('Loan_ID',1,inplace=True)
        df1=data_frame.drop(['Dependents','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term',
                             'Loan_Status','IncomePerDep','LoanPerYear','Residual','IncomeRatio'],1)
        df2=data_frame[['Dependents','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term',
                        'Loan_Status','IncomePerDep','LoanPerYear','Residual','IncomeRatio']]
    else:
        data_frame.drop('Loan_ID',1,inplace=True)
        df1=data_frame.drop(['Dependents','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term',
                             'IncomePerDep','LoanPerYear','Residual','IncomeRatio'],1)
        df2=data_frame[['Dependents','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term',
                        'IncomePerDep','LoanPerYear','Residual','IncomeRatio']]
        
    #print(data_frame.head())
    df1=df1.apply(LabelEncoder().fit_transform)
    data_frame=pd.concat([df1,df2],1)
    return data_frame

def scaling_func(data_frame,train=True):
    scaler = preprocessing.MinMaxScaler()
    if train:
        scaled_df = scaler.fit_transform(data_frame.drop('Loan_Status',1))
    else:
        scaled_df = scaler.fit_transform(data_frame)
    scaled_df = pd.DataFrame(scaled_df, columns=['Gender','Married','Education','Self_Employed',
                                                 'Credit_History','Property_Area','LoanByIncome',
                                                 'Dependents','ApplicantIncome',
                                                 'CoapplicantIncome','LoanAmount','Loan_Amount_Term',
                                                 'IncomePerDep','LoanPerYear','Residual','IncomeRatio'])
    return scaled_df

def filet(data_frame):
    filtered_df=data_frame[['Married','Education','Credit_History','Property_Area',
                           'CoapplicantIncome','LoanByIncome','LoanPerYear','Residual','Dependents']]
    return filtered_df

def boosting_data_frame(df,size=614):
    list1=np.random.choice(614, 614,replace=True)
    return df.iloc[list1]

In [308]:
df=pre_processing(df)

In [309]:
df.head()

,Gender,Married,Education,Self_Employed,Credit_History,Property_Area,LoanByIncome,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Loan_Status,IncomePerDep,LoanPerYear,Residual,IncomeRatio
0,1,0,0,0,1,2,15,0.0,5849,0.0,-1.0,360.0,Y,2924.500000,-33.333333,5882.333333,5849.000000
1,1,2,0,0,1,0,251,1.0,4583,1508.0,128.0,360.0,N,1527.666667,4266.666667,1824.333333,46.853846
2,1,2,0,2,1,2,137,0.0,3000,0.0,66.0,360.0,Y,1500.000000,2200.000000,800.000000,44.117647
3,1,2,1,0,1,2,499,0.0,2583,2358.0,120.0,360.0,Y,1291.500000,4000.000000,941.000000,40.500000
4,1,0,0,0,1,2,167,0.0,6000,0.0,141.0,360.0,Y,3000.000000,4700.000000,1300.000000,41.958042


In [310]:
df.columns

Index(['Gender', 'Married', 'Education', 'Self_Employed', 'Credit_History',
       'Property_Area', 'LoanByIncome', 'Dependents', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Loan_Status',
       'IncomePerDep', 'LoanPerYear', 'Residual', 'IncomeRatio'],
      dtype='object')

In [311]:
df_scaled=scaling_func(df)

In [313]:
df_scaled.head()

,Gender,Married,Education,Self_Employed,Credit_History,Property_Area,LoanByIncome,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,IncomePerDep,LoanPerYear,Residual,IncomeRatio
0,0.5,0.0,0.0,0.0,1.0,1.0,0.024712,0.25,0.070489,0.000000,0.000000,0.75052,0.090840,0.951619,0.047682,0.282595
1,0.5,1.0,0.0,0.0,1.0,0.0,0.413509,0.50,0.054830,0.036192,0.184023,0.75052,0.046659,0.953493,0.045913,0.001688
2,0.5,1.0,0.0,1.0,1.0,1.0,0.225700,0.25,0.035250,0.000000,0.095578,0.75052,0.045784,0.952593,0.045467,0.001555
3,0.5,1.0,1.0,0.0,1.0,1.0,0.822076,0.25,0.030093,0.056592,0.172611,0.75052,0.039189,0.953377,0.045528,0.001380
4,0.5,0.0,0.0,0.0,1.0,1.0,0.275124,0.25,0.072356,0.000000,0.202568,0.75052,0.093228,0.953682,0.045684,0.001451


In [314]:
X=df_scaled
y=df['Loan_Status']

In [315]:
# Select features with highest chi-squared statistics
chi2_selector = SelectKBest(chi2, k=16)
X_kbest = chi2_selector.fit_transform(X, y)

mask = chi2_selector.get_support()

new_features = X.columns[mask]
new_features

Index(['Gender', 'Married', 'Education', 'Self_Employed', 'Credit_History',
       'Property_Area', 'LoanByIncome', 'Dependents', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'IncomePerDep',
       'LoanPerYear', 'Residual', 'IncomeRatio'],
      dtype='object')

In [316]:
chi2_selector.scores_[mask]

array([4.90805074e-03, 1.65626180e+00, 3.54050246e+00, 9.97562828e-04,
       2.60148043e+01, 1.88918732e-01, 5.82833294e-02, 1.95700058e-02,
       1.19463446e-03, 2.72206821e-01, 5.26026627e-03, 1.39699229e-03,
       6.24726090e-03, 5.14914221e-03, 8.63285156e-02, 9.19156115e-01])

In [317]:
chi2_selector.pvalues_[mask]

array([9.44147908e-01, 1.98108805e-01, 5.98873168e-02, 9.74803629e-01,
       3.38809315e-07, 6.63818075e-01, 8.09230074e-01, 8.88744661e-01,
       9.72427819e-01, 6.01855245e-01, 9.42181968e-01, 9.70184921e-01,
       9.37001113e-01, 9.42794877e-01, 7.68897632e-01, 3.37696647e-01])

In [318]:
feature_array=pd.DataFrame({'new_features':new_features,'scores':chi2_selector.scores_[mask],'pvalues':chi2_selector.pvalues_[mask]})
feature_array.sort_values('scores',ascending=False)

,new_features,scores,pvalues
4,Credit_History,26.014804,3.388093e-07
2,Education,3.540502,5.988732e-02
1,Married,1.656262,1.981088e-01
15,IncomeRatio,0.919156,3.376966e-01
9,CoapplicantIncome,0.272207,6.018552e-01
5,Property_Area,0.188919,6.638181e-01
14,Residual,0.086329,7.688976e-01
6,LoanByIncome,0.058283,8.092301e-01
7,Dependents,0.019570,8.887447e-01
12,IncomePerDep,0.006247,9.370011e-01


In [319]:
filtered_df=filet(df_scaled)
filtered_df['Loan_Status']=df['Loan_Status']
filtered_df.head()

C:\Users\irwing\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Married,Education,Credit_History,Property_Area,CoapplicantIncome,LoanByIncome,LoanPerYear,Residual,Dependents,Loan_Status
0,0.0,0.0,1.0,1.0,0.000000,0.024712,0.951619,0.047682,0.25,Y
1,1.0,0.0,1.0,0.0,0.036192,0.413509,0.953493,0.045913,0.50,N
2,1.0,0.0,1.0,1.0,0.000000,0.225700,0.952593,0.045467,0.25,Y
3,1.0,1.0,1.0,1.0,0.056592,0.822076,0.953377,0.045528,0.25,Y
4,0.0,0.0,1.0,1.0,0.000000,0.275124,0.953682,0.045684,0.25,Y


In [321]:
df1=boosting_data_frame(filtered_df)
X=np.array(df1.drop('Loan_Status',axis=1))
y=np.array(df1['Loan_Status'])

In [322]:
# Setup the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
penalty_space=['l1','l2']
param_grid = {'C': c_space,'penalty':penalty_space}

# Instantiate a logistic regression classifier: clf
clf = LogisticRegression()

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(clf, param_grid, cv=5)

# Fit it to the data
logreg_cv.fit(X, y)

# Print the tuned parameter and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_))
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 0.05179474679231213, 'penalty': 'l1'}
Best score is 0.758957654723127


In [323]:
df1=boosting_data_frame(filtered_df)
X=np.array(df1.drop('Loan_Status',axis=1))
y=np.array(df1['Loan_Status'])

In [324]:
# Setup the hyperparameter grid
n_estimators_space = [10,50,100]
criterion_space=['gini','entropy']
min_samples_leaf_space=[1,10,50,100]
max_features_space=['auto','sqrt','log2',0.2]
param_grid = {'n_estimators': n_estimators_space,'criterion':criterion_space,'min_samples_leaf':min_samples_leaf_space,
             'max_features':max_features_space}

# Instantiate a logistic regression classifier: clf
clf = RandomForestClassifier(oob_score=True)

# Instantiate the GridSearchCV object: logreg_cv
RF_cv = GridSearchCV(clf, param_grid, cv=5)

# Fit it to the data
RF_cv.fit(X, y)

# Print the tuned parameter and score
print("Tuned Random Forest Parameters: {}".format(RF_cv.best_params_))
print("Best score is {}".format(RF_cv.best_score_))

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probab

Tuned Logistic Regression Parameters: {'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 50}
Best score is 0.8876221498371335


In [325]:
df1=boosting_data_frame(filtered_df)
X=np.array(df1.drop('Loan_Status',axis=1))
y=np.array(df1['Loan_Status'])

In [326]:
# Setup the hyperparameter grid
n_neighbors_space = [1,5,10,20,50,100,150]
weights_space=['uniform','distance']

param_grid = {'n_neighbors': n_neighbors_space,'weights':weights_space}

# Instantiate a logistic regression classifier: clf
clf = KNeighborsClassifier()

# Instantiate the GridSearchCV object: logreg_cv
KNN_cv = GridSearchCV(clf, param_grid, cv=5)

# Fit it to the data
KNN_cv.fit(X, y)

# Print the tuned parameter and score
print("Tuned KNN Parameters: {}".format(KNN_cv.best_params_))
print("Best score is {}".format(KNN_cv.best_score_))

Tuned Logistic Regression Parameters: {'n_neighbors': 100, 'weights': 'distance'}
Best score is 0.8827361563517915


In [327]:
df1=boosting_data_frame(filtered_df)
X=np.array(df1.drop('Loan_Status',axis=1))
y=np.array(df1['Loan_Status'])

In [328]:
Log_pred=logreg_cv.predict_proba(X)[:,1] #we only want probability of one class
RF_pred=RF_cv.predict_proba(X)[:,1]
KNN_pred=KNN_cv.predict_proba(X)[:,1]

In [329]:
new_df=pd.DataFrame({'Log_pred':Log_pred,'RF_pred':RF_pred,'KNN_pred':KNN_pred})
new_df['Class']=y
new_df.head()

,Log_pred,RF_pred,KNN_pred,Class
0,0.749643,0.98,0.947447,Y
1,0.721742,0.80,1.000000,Y
2,0.721742,0.24,0.000000,N
3,0.500000,1.00,1.000000,Y
4,0.749643,0.90,1.000000,Y


In [330]:
X=new_df.drop('Class',axis=1)
y=new_df['Class']

In [331]:
# Setup the hyperparameter grid
max_depth_space=[1,3,10,20,50,100]
n_estimators_space = [10,50,100]

param_grid = {'n_estimators': n_estimators_space,'max_depth':max_depth_space}

# Instantiate a logistic regression classifier: clf
clf = xgboost.XGBClassifier()

# Instantiate the GridSearchCV object: logreg_cv
XGB_cv = GridSearchCV(clf, param_grid, cv=5)

# Fit it to the data
XGB_cv.fit(X, y)

# Print the tuned parameter and score
print("Tuned XGB Parameters: {}".format(XGB_cv.best_params_))
print("Best score is {}".format(XGB_cv.best_score_))

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

Tuned Logistic Regression Parameters: {'max_depth': 3, 'n_estimators': 10}
Best score is 0.9788273615635179


C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

In [333]:
df_test=pd.read_csv("test_Y3wMUE5_7gLdaTN.csv")
df_test2=df_test.copy()
df_new=pre_processing(df_test2,False)

In [334]:
df_scaled_new=scaling_func(df_new,False)
X=np.array(filet(df_scaled_new))

In [335]:
Log_pred=logreg_cv.predict_proba(X)[:,1] #we only want probability of one class
RF_pred=RF_cv.predict_proba(X)[:,1]
KNN_pred=KNN_cv.predict_proba(X)[:,1]

new_df=pd.DataFrame({'Log_pred':Log_pred,'RF_pred':RF_pred,'KNN_pred':KNN_pred})
new_df['Class']=y
new_df.head()

X=new_df.drop('Class',axis=1)
y=new_df['Class']

df_test['Loan_Status']=Top_Clf.predict(X)

C:\Users\irwing\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [336]:
df_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,Y
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,Y
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,Y
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,Y
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,Y


In [337]:
df_test.to_csv('submission_ensemble.csv',columns=(['Loan_ID','Loan_Status']),index=False,header=(['Loan_ID','Loan_Status']))